In [1]:
import pandas as pd
import backtrader as bt
from datetime import datetime



In [2]:
# Load the data and convert the timestamp to datetime
df = pd.read_csv('../DB/bitget/1m/FET_USDT_USDT.csv') 

# Convert 'date' to datetime format
df['date'] = pd.to_datetime(df['date'], unit='ms')  # Assuming the 'date' is in milliseconds

# Set 'date' as index
df.set_index('date', inplace=True)

df

,open,high,low,close,volume,usdtVolume
date,,,,,,
1970-01-20 06:02:08.040,0.08780,0.08780,0.08740,0.08740,752,65.86840
1970-01-20 06:02:08.100,0.08740,0.08790,0.08740,0.08790,544,47.68090
1970-01-20 06:02:08.160,0.08790,0.08800,0.08750,0.08760,1063,93.18800
1970-01-20 06:02:08.220,0.08760,0.08800,0.08740,0.08790,546,47.90620
1970-01-20 06:02:08.280,0.08790,0.08790,0.08740,0.08750,603,52.83270
...,...,...,...,...,...,...
1970-01-20 06:02:23.700,0.08742,0.08748,0.08718,0.08743,557,48.64813
1970-01-20 06:02:23.760,0.08743,0.08747,0.08711,0.08742,594,51.83305
1970-01-20 06:02:23.820,0.08742,0.08746,0.08716,0.08723,547,47.75409


In [3]:
df

,open,high,low,close,volume,usdtVolume
date,,,,,,
1970-01-20 06:02:08.040,0.08780,0.08780,0.08740,0.08740,752,65.86840
1970-01-20 06:02:08.100,0.08740,0.08790,0.08740,0.08790,544,47.68090
1970-01-20 06:02:08.160,0.08790,0.08800,0.08750,0.08760,1063,93.18800
1970-01-20 06:02:08.220,0.08760,0.08800,0.08740,0.08790,546,47.90620
1970-01-20 06:02:08.280,0.08790,0.08790,0.08740,0.08750,603,52.83270
...,...,...,...,...,...,...
1970-01-20 06:02:23.700,0.08742,0.08748,0.08718,0.08743,557,48.64813
1970-01-20 06:02:23.760,0.08743,0.08747,0.08711,0.08742,594,51.83305
1970-01-20 06:02:23.820,0.08742,0.08746,0.08716,0.08723,547,47.75409


In [8]:

# Define the strategy
class MultiLevelBandStrategy(bt.SignalStrategy):
    params = (
        ('length', 20),
        ('mult1', 0.5),
    )
    
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.length)
        self.stdev = bt.indicators.StandardDeviation(self.data.close, period=self.params.length)
        
        self.level1_above = self.sma + self.stdev * self.params.mult1
        self.level1_below = self.sma - self.stdev * self.params.mult1
        
        self.buy_signal = bt.indicators.CrossOver(self.data.close, self.level1_below)
        self.sell_signal = bt.indicators.CrossDown(self.data.close, self.level1_above)
    
    def next(self):
        print(f"Close: {self.data.close[0]}")
        print(f"Level1 Below: {self.level1_below[0]}")
        print(f"Level1 Above: {self.level1_above[0]}")
        
        if self.data.close[0] > self.level1_below[0]:
            self.buy()
        elif self.data.close[0] < self.level1_above[0]:
            self.sell()

# Define the PandasData feed
class PandasData(bt.feeds.PandasData):
    lines = ('volume',)
    params = (('volume', -1),)

# Create cerebro instance
cerebro = bt.Cerebro()
cerebro.addstrategy(MultiLevelBandStrategy)

# Create and add data feed
data_feed = PandasData(dataname=df)
cerebro.adddata(data_feed)

# Set initial cash and commission
cerebro.broker.set_cash(10000)
cerebro.broker.setcommission(commission=0.001)

# Run the backtest
results = cerebro.run()

# Plot results
figs = cerebro.plot(style='candlestick')
plt.show()

Close: 0.0886
Level1 Below: 0.08793410321687992
Level1 Above: 0.08818589678312008
Close: 0.0884
Level1 Below: 0.08795531287650608
Level1 Above: 0.08821468712349391
Close: 0.0884
Level1 Below: 0.08800367914441334
Level1 Above: 0.08824632085558665
Close: 0.08815
Level1 Below: 0.08801894753271425
Level1 Above: 0.08825605246728577
Close: 0.08816
Level1 Below: 0.0880771796511992
Level1 Above: 0.0882638203488008
Close: 0.08807
Level1 Below: 0.08809989589783998
Level1 Above: 0.08826810410216002
Close: 0.08843
Level1 Below: 0.08812897584713664
Level1 Above: 0.08829202415286336
Close: 0.08828
Level1 Below: 0.08814637390000718
Level1 Above: 0.08830262609999281
Close: 0.08813
Level1 Below: 0.08814222976450714
Level1 Above: 0.08829977023549287
Close: 0.08813
Level1 Below: 0.08814423961730361
Level1 Above: 0.08830076038269641
Close: 0.08818
Level1 Below: 0.08813863352775404
Level1 Above: 0.08829436647224596
Close: 0.0883
Level1 Below: 0.08814932689911616
Level1 Above: 0.08830367310088384
Close: 0.0

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 7 Axes>]]

In [10]:
fig = cerebro.plot()[0][0]
fig.savefig('backtest_result.png', dpi=300)

<IPython.core.display.Javascript object>